This is an extension of previous notebook. If you are in a hurry you can read between this- 
"

MLflow runs can be recorded to
1. local files
2. to a SQLAlchemy compatible database, 
3. remotely to a tracking server.
By default, the MLflow Python API logs runs locally to files in an mlruns directory wherever you ran your program. You can then run mlflow ui to see the logged runs.

MLflow uses two components for storage: backend store and artifact store.
While the **backend store** records runs, model's parameters, metrics, tags, notes, metadata, etc), the **artifact store** records artifacts like (files, models, images, in-memory objects, or model summary, etc).

"



MLflow setup:

**tracking server**: yes, local server

**backend store**: sqlite database

**artifacts store**: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///mlflow_local_sqlite.db --default-artifact-root ./mlruns

In [1]:

import mlflow
import warnings
warnings.filterwarnings('ignore')

In [3]:
mlflow.list_experiments()


[<Experiment: artifact_location='file:///home/shivam/MLflow_examples/Experiment_tracking/mlruns/1', experiment_id='1', lifecycle_stage='active', name='experiment-local-run', tags={}>,
 <Experiment: artifact_location='file:///home/shivam/MLflow_examples/Experiment_tracking/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='file:///home/shivam/MLflow_examples/Experiment_tracking/mlruns/2', experiment_id='2', lifecycle_stage='active', name='experiment-local-run-diff-flavor-model', tags={}>]

In [7]:
mlflow.set_tracking_uri("sqlite:///mlflow_local_sqlite.db")
mlflow.set_experiment("experiment-local-sql")
# set_tracking_uri is used to point to our backend. The provided uri should match the uri you provided when you run the MLflow command from console.
# set_experiment is used to define the name of our experiments. You can use whatever is convenient for you.

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='experiment-local-sql', tags={}>

In [8]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='experiment-local-sql', tags={}>]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# mlflow.set_experiment("my-experiment-3")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: './mlruns/1/fb673d2e035844c0a0649a0251f5b78d/artifacts'


In [13]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='experiment-local-sql', tags={}>]

In [14]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [17]:
from mlflow.exceptions import MlflowException

try:
    print (client.list_registered_models())
except MlflowException:
    print("It's not possible to access the model registry :(")

[]


In [18]:
"schedule another run now"
with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.001, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: './mlruns/1/61690f6cdefb4184b769ead07dc1fc68/artifacts'
